In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [144]:
from sklearn import set_config
set_config(display='diagram')

In [195]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [202]:
df = pd.read_csv('../project_2/customer_churn_dataset-training-master.csv')

In [203]:
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [204]:
df = df.drop(columns=['CustomerID'],axis=1)

In [207]:
df = df.dropna()

In [209]:
df.isnull().sum()

Age                  0
Gender               0
Tenure               0
Usage Frequency      0
Support Calls        0
Payment Delay        0
Subscription Type    0
Contract Length      0
Total Spend          0
Last Interaction     0
Churn                0
dtype: int64

In [212]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Churn'], axis=1), 
                                                 df['Churn'], test_size=0.2, 
                                                 random_state=42)

In [213]:
# x_train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [214]:
# # imputation transformer
# trf1 = ColumnTransformer([
#     ('impute_age',SimpleImputer(),[0,2,3,4,5,8,9]),
#     ('impute_embarked',SimpleImputer(strategy='most_frequent'),[1,6,7])
# ],remainder='passthrough')

In [215]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6,7])
],remainder='passthrough')

In [216]:
for i,j in enumerate(x_train):
    print(i,j," ",x_train[j].dtype)

0 Age   float64
1 Gender   object
2 Tenure   float64
3 Usage Frequency   float64
4 Support Calls   float64
5 Payment Delay   float64
6 Subscription Type   object
7 Contract Length   object
8 Total Spend   float64
9 Last Interaction   float64


In [217]:
# # Scaling
# trf3 = ColumnTransformer([
#     ('scale',MinMaxScaler(),slice(0,10))
# ])

In [218]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [219]:
# train the model
trf5 = RandomForestClassifier() 

In [226]:
pipe = Pipeline([
#     ('trf1',trf1),
    ('trf2',trf2),
#     ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [227]:
# train
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6, 7])])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f9b5b403b80>)),
                ('trf5', RandomForestClassifier())])

In [228]:
y_pred = pipe.predict(x_test)

In [229]:
from sklearn import metrics

In [230]:
metrics.accuracy_score(y_test,y_pred)

0.987954676919936

In [231]:
metrics.confusion_matrix(y_test,y_pred)

array([[38059,     4],
       [ 1058, 49046]])